<a href="https://colab.research.google.com/github/peppopi/Piratas/blob/main/Order%2066%20-%20R34.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown ### 5️⃣ Dependencias (Dale Clik a este triangulo)
!pip install rule34Py
!pip install python-resize-image
!pip install mplcursors
!pip install ipyplot
!pip install ipyvuetify
!pip install ipyevents
!pip install ipyevents
!pip install dash
!pip install dash_core_components
!pip install dash_html_componentsimport os

In [ ]:
#@markdown ### 5️⃣ Programa de descarga
import os
import requests
from rule34Py import rule34Py
import shutil
# Crear una instancia de rule34Py
r34Py = rule34Py()
#@markdown Donde guardar las imagenes:
# Carpeta de destino para las imágenes
folder_path = '/content/images' #@param {type:"string"}
#@markdown Ej: /content/images Se guardara en la carpeta imagenes del entorno virtual, es decir no ocupara espacio (No hay necesidad de usar google drive)
#@markdown
#@markdown Ej: /content/drive/MyDrive/Dataset Se guardaran en tu drive, por lo que no tendras que descargarlas despues, pero si ocuparan espacio (Debes conetar tu drive si queres usarla)
os.makedirs(folder_path, exist_ok=True)
#@markdown
#@markdown
#@markdown Aca pones el tag que buscas , verificar que este bien escrito
Tag = "pokemon " #@param {type:"string"}
#@markdown Aca pones la cantidad de imagenes a descargar, maximo mil
Maximo = 5 #@param {type:"number"}
# Realizar la búsqueda de imágenes
search_results = r34Py.search([Tag], limit=Maximo)

# Descargar y guardar las imágenes
for result in search_results:
    image_url = result.image
    image_id = result.id

    # Obtener el nombre del archivo y su extensión
    _, ext = os.path.splitext(image_url)

    # Construir la ruta de destino para la imagen
    image_path = os.path.join(folder_path, f'{image_id}{ext}')

    # Descargar la imagen desde la URL
    response = requests.get(image_url, stream=True)
    if response.status_code == 200:
        with open(image_path, 'wb') as f:
            response.raw.decode_content = True
            shutil.copyfileobj(response.raw, f)
        print(f'Imagen guardada: {image_path}')
    else:
        print(f'Error al descargar la imagen: {image_url}')


In [ ]:
#@markdown ### 5️⃣ Vizualizaor Low Data
import ipywidgets as widgets
from IPython.display import display
from PIL import Image

#@markdown Ruta de la carpeta donde se guardan las imágenes descargadas
folder_path = '/content/images' #@param {type:"string"}
# Obtener la lista de archivos en la carpeta
files = os.listdir(folder_path)

# Crear una lista de widgets de imagen para cada archivo
image_widgets = []
for file_name in files:
    image_path = os.path.join(folder_path, file_name)

    # Redimensionar la imagen si es necesario
    with Image.open(image_path) as img:
        if img.size[0] > 1200 or img.size[1] > 1200:
            img.thumbnail((1200, 1200), Image.ANTIALIAS)

    # Crear el widget de imagen y agregarlo a la lista
    image_widget = widgets.Image(value=open(image_path, 'rb').read())
    image_widgets.append(image_widget)

# Crear un contenedor de galería con VBox
gallery = widgets.VBox(children=image_widgets)

# Mostrar la galería
display(gallery)




In [ ]:
#@markdown ### 5️⃣ Visualizador big data
import cv2
import matplotlib.pyplot as plt
import os

#@markdown Ruta de la carpeta donde se guardan las imágenes descargadas
folder_path = '/content/images' #@param {type:"string"}

# Obtener la lista de archivos en la carpeta
files = os.listdir(folder_path)

#@markdown Configurar el tamaño de la cuadrícula 10 X 10 = maximo 100 imagenes
num_rows = 5 #@param {type:"number"}
num_cols = 5 #@param {type:"number"}

# Crear una figura y una cuadrícula de subtramas
fig, axs = plt.subplots(num_rows, num_cols, figsize=(20, 20))

# Iterar sobre los archivos de imágenes y mostrarlos en las subtramas
for i, file in enumerate(files):
    # Leer la imagen usando OpenCV
    image_path = os.path.join(folder_path, file)
    img = cv2.imread(image_path)

    # Redimensionar la imagen si es necesario
    if img.shape[0] > 1200 or img.shape[1] > 1200:
        img = cv2.resize(img, (1200, 1200))

    # Mostrar la imagen en la subtrama correspondiente
    row = i // num_cols
    col = i % num_cols
    axs[row, col].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    axs[row, col].axis('off')

    # Agregar una etiqueta de texto debajo de la imagen
    delete_label = axs[row, col].text(0.5, -0.1, 'Fbi Alert', transform=axs[row, col].transAxes,
                                      ha='center', va='top', color='red', fontsize=12)

    # Función para borrar la imagen al hacer clic en la etiqueta de texto
    def delete_image(event):
        if event.inaxes == axs[row, col]:
            # Eliminar la imagen del archivo
            os.remove(image_path)
            print(f"Imagen eliminada: {file}")
            # Actualizar la visualización para reflejar la eliminación
            axs[row, col].cla()
            axs[row, col].axis('off')
            axs[row, col].text(0.5, -0.1, 'Fbi Alert', transform=axs[row, col].transAxes,
                               ha='center', va='top', color='red', fontsize=12)
            plt.draw()

    # Conectar la función de eliminación al evento de clic en la etiqueta de texto
    delete_label.set_picker(True)
    fig.canvas.mpl_connect('pick_event', delete_image)

# Ajustar los espacios entre las subtramas
plt.tight_layout()

# Mostrar la figura
plt.show()



In [ ]:
#@markdown ### 5️⃣ Visualizador big data (me)

import dash
from dash import dcc, html
import base64
from PIL import Image
import io

#@markdown Ruta de la carpeta donde se guardan las imágenes descargadas
folder_path = '/content/images' #@param {type:"string"}

# Obtener la lista de archivos en la carpeta
files = os.listdir(folder_path)

# Crear una lista para almacenar las imágenes
images = []

# Iterar sobre los archivos de imágenes y cargarlas
for file in files:
    # Cargar la imagen utilizando PIL
    image_path = os.path.join(folder_path, file)
    img = Image.open(image_path)

    # Convertir la imagen a formato base64
    img_data = io.BytesIO()
    img.save(img_data, format='PNG')
    img_base64 = base64.b64encode(img_data.getvalue()).decode('utf-8')

    # Agregar el formato base64 a la lista de imágenes
    images.append(img_base64)

# Crear la aplicación Dash
app = dash.Dash(__name__)

# Definir el diseño de la aplicación
app.layout = html.Div([
    html.H1("Cuadrícula de imágenes"),
    html.Div(id="image-grid", style={"display": "grid", "grid-template-columns": "repeat(10, 1fr)", "grid-gap": "10px"})
])

# Función para generar la cuadrícula de imágenes
def generate_image_grid():
    # Crear la lista de elementos HTML para mostrar las imágenes en una cuadrícula
    items = []
    for img_base64 in images:
        # Crear el elemento HTML para mostrar la imagen
        img_html = html.Img(src=f"data:image/png;base64,{img_base64}", width="200", height="200")
        items.append(img_html)

    return items

# Actualizar la cuadrícula de imágenes cuando se carga la página
@app.callback(
    dash.dependencies.Output("image-grid", "children"),
    dash.dependencies.Input("image-grid", "id")
)
def update_image_grid(id):
    return generate_image_grid()

# Ejecutar la aplicación
if __name__ == "__main__":
    app.run_server()



In [ ]:
#@markdown ### 5️⃣ Eiminador Big Data, magic code No Gpu
import os
import ipywidgets as widgets
from IPython.display import display
from PIL import Image

# Ruta de la carpeta donde se guardan las imágenes
folder_path = '/content/images'

# Obtener la lista de archivos en la carpeta
files = os.listdir(folder_path)

# Crear una lista para almacenar las imágenes y sus contenedores
images = []
image_containers = []

# Cargar las imágenes y crear los contenedores correspondientes
for file in files:
    image_path = os.path.join(folder_path, file)
    image = Image.open(image_path)
    image.thumbnail((200, 200))
    image_widget = widgets.Image(value=image._repr_png_(), format='png', width=200, height=200)
    remove_button = widgets.Button(description='Eliminar', button_style='danger')
    image_container = widgets.VBox(children=[image_widget, remove_button], layout=widgets.Layout(align_items='center'))
    image_container.file = file
    images.append(image_container)
    image_containers.append(image_container)

# Función para eliminar una imagen
def remove_image(button):
    for i, image_container in enumerate(list(grid.children)):
        if image_container.children[1] == button:
            container = grid.children[i]
            grid.children = tuple(widget for widget in grid.children if widget != container)
            images.remove(image_container)
            # Eliminar la imagen del directorio
            image_file = os.path.join(folder_path, image_container.file)
            os.remove(image_file)
            break

# Crear una cuadrícula para mostrar las imágenes
grid = widgets.GridBox(layout=widgets.Layout(grid_template_columns="repeat(4, 250px)"))

# Agregar las imágenes y los botones a la cuadrícula
for image_container in image_containers:
    remove_button = image_container.children[1]
    remove_button.on_click(remove_image)
    grid.children += (image_container,)

# Mostrar la cuadrícula de imágenes
display(grid)



In [23]:
#@markdown ### 5️⃣ Descarga de alta velocidad
import cv2
import os
import zipfile

#@markdown Ruta del directorio de imágenes
directorio_imagenes = '/content/images' #@param {type:"string"}

#@markdown Por sentido común se establece un tamaño máximo deseado para las imágenes de 1200 x 1200, en caso contrario se redimensionan al máximo
Altura = 1200 #@param {type:"number"}
Base = 1200 #@param {type:"number"}
tamano_maximo = (Base, Altura)

# Listas para almacenar las rutas de las imágenes redimensionadas y originales
imagenes_redimensionadas = []
imagenes_originales = []

# Recorre todos los archivos en el directorio de imágenes
for archivo in os.listdir(directorio_imagenes):
    ruta_archivo = os.path.join(directorio_imagenes, archivo)

    # Verifica si el archivo es una imagen
    if os.path.isfile(ruta_archivo) and archivo.lower().endswith(('.jpg', '.jpeg', '.png', '.gif')):
        imagen = cv2.imread(ruta_archivo)

        # Obtiene las dimensiones de la imagen
        altura, ancho = imagen.shape[:2]

        # Verifica si la imagen es mayor al tamaño máximo
        if altura > tamano_maximo[0] or ancho > tamano_maximo[1]:
            # Calcula las nuevas dimensiones proporcionales
            proporcion = min(tamano_maximo[0] / altura, tamano_maximo[1] / ancho)
            nueva_altura = int(altura * proporcion)
            nueva_ancho = int(ancho * proporcion)

            # Redimensiona la imagen
            imagen_redimensionada = cv2.resize(imagen, (nueva_ancho, nueva_altura))

            # Guarda la imagen redimensionada en una nueva ruta
            ruta_redimensionada = os.path.join(directorio_imagenes, f"redimensionada_{archivo}")
            cv2.imwrite(ruta_redimensionada, imagen_redimensionada)

            # Agrega la ruta de la imagen redimensionada a la lista correspondiente
            imagenes_redimensionadas.append(ruta_redimensionada)
        else:
            # Agrega la ruta de la imagen original a la lista correspondiente
            imagenes_originales.append(ruta_archivo)

# Crea un archivo zip y agrega las imágenes redimensionadas
archivo_zip = '/content/outputs.zip'
with zipfile.ZipFile(archivo_zip, 'w') as zip:
    for ruta_imagen in imagenes_redimensionadas:
        archivo_zip_nombre = os.path.basename(ruta_imagen)
        zip.write(ruta_imagen, archivo_zip_nombre)

# Descarga el archivo zip
from google.colab import files
files.download(archivo_zip)



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [25]:
#@markdown ### 5️⃣ Descarga sin redimencionado
import os
import zipfile

#@markdown Ruta del directorio de imágenes Ruta del directorio de imágenes
directorio_imagenes = '/content/images' #@param {type:"string"}

# Crea un archivo zip y agrega las imágenes en el directorio
archivo_zip = '/content/outputs.zip'
with zipfile.ZipFile(archivo_zip, 'w') as zip:
    for archivo in os.listdir(directorio_imagenes):
        ruta_archivo = os.path.join(directorio_imagenes, archivo)

        # Verifica si el archivo es una imagen
        if os.path.isfile(ruta_archivo) and archivo.lower().endswith(('.jpg', '.jpeg', '.png', '.gif')):
            zip.write(ruta_archivo, archivo)

# Descarga el archivo zip
from google.colab import files
files.download(archivo_zip)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#@markdown Inicie el Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
if "step1_installed_flag" not in globals():
  raise Exception("Por favor usa el paso 1 primero.")

#@markdown ### 🚮 Eliminar Gif
#@markdown Cuidado, borra todos los archivos que no sean imágenes de la carpeta del proyecto.

!find {images_folder} -type f ! \( -name '*.png' -o -name '*.jpg' -o -name '*.jpeg' \) -delete